In [ ]:
from google.colab import files, drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/BUDT758B_FinalProject')


### Reshaping Dataset

In [ ]:
# Read the data into a pandas data frame:
import json
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

beauty = getDF('All_Beauty.json.gz')
beauty.head(10)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"09 1, 2016",A3CIUOJXQ5VDQ2,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Shelly F,As advertised. Reasonably priced,Five Stars,1472688000,NaN,NaN
1,5.0,True,"11 14, 2013",A3H7T87S984REU,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",houserules18,Like the oder and the feel when I put it on my...,Good for the face,1384387200,NaN,NaN
2,1.0,True,"08 18, 2013",A3J034YH7UG4KT,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Adam,I bought this to smell nice after I shave. Wh...,Smells awful,1376784000,NaN,NaN
3,5.0,False,"05 3, 2011",A2UEO5XR3598GI,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Rich K,HEY!! I am an Aqua Velva Man and absolutely lo...,Truth is There IS Nothing Like an AQUA VELVA MAN.,1304380800,25,NaN
4,5.0,True,"05 6, 2011",A3SFRT223XXWF7,B00006L9LC,{'Size:': ' 200ml/6.7oz'},C. C. Christian,If you ever want to feel pampered by a shampoo...,Bvlgari Shampoo,1304640000,3,NaN
5,5.0,False,"05 16, 2010",A24HQ2N7332W7W,B00006L9LC,{'Size:': ' 366'},Kindle Customer Joyce Wilson,"If you know the scent of Diva, you'll LOVE thi...",Diva is Heavenly,1273968000,NaN,NaN
6,5.0,False,"05 7, 2018",A2G90R2ZU6KU5D,B00006L9LC,{'Size:': ' Small'},Mike,Got this shampoo as a solution for my wife's d...,"Outstanding, no complains",1525651200,NaN,NaN
7,2.0,True,"05 7, 2018",A24W4W9E62FZP2,B00006L9LC,{'Size:': ' Small'},Reb,No change my scalp still itches like crazy. It...,No change my scalp still itches like crazy. It...,1525651200,NaN,NaN
8,1.0,True,"05 6, 2018",A7ID5H7FWLJHC,B00006L9LC,{'Size:': ' Small'},U. V.,Too expensive for such poor quality. There was...,Too expensive for such poor quality. There was...,1525564800,NaN,NaN
9,1.0,True,"05 6, 2018",AYKOSAJTP5AVS,B00006L9LC,{'Size:': ' Small'},Senthil Kumar M,"It dries my hair, doesnt help to reduce dandru...","Dries my hair, doesnt help to reduce dandruff....",1525564800,NaN,NaN


In [ ]:
import numpy as np
beauty.overall.unique()
beauty['attitude'] = np.where((beauty.overall > 3), 'pos', 'neg')
beauty_filtered = beauty[['reviewText','attitude']]
beauty_filtered

,reviewText,attitude
0,As advertised. Reasonably priced,pos
1,Like the oder and the feel when I put it on my...,pos
2,I bought this to smell nice after I shave. Wh...,neg
3,HEY!! I am an Aqua Velva Man and absolutely lo...,pos
4,If you ever want to feel pampered by a shampoo...,pos
...,...,...
5264,I have genetic undereye darkness. Ive accepted...,pos
5265,I absolutely love this eye gel.,pos
5266,The eye gel is easy to apply and I use it morn...,pos
5267,Ok this eye gel is good stuff.,pos


In [ ]:
beauty_pos = beauty_filtered[beauty_filtered['attitude']=='pos']
beauty_neg = beauty_filtered[beauty_filtered['attitude']=='neg']

print("pos shape:" + str(beauty_pos.shape))
print("neg shape:" + str(beauty_neg.shape))

for i in range(4981):
  save_path = '/content/drive/My Drive/BUDT758B_FinalProject/pos/'
  value = str(beauty_pos.iloc[i]['reviewText'])
  filename = os.path.join(save_path, 'pos' + str(10000+i) +'.txt')
  f = open(filename,"w")
  f.write(value)
  f.close

for i in range(288):
  save_path = '/content/drive/My Drive/BUDT758B_FinalProject/neg/'
  value = str(beauty_neg.iloc[i]['reviewText'])
  filename = os.path.join(save_path, 'neg' + str(10000+i) +'.txt')
  f = open(filename,"w")
  f.write(value)
  f.close


### Data Processing

In [ ]:
import os, sys
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
import collections
import statistics

pos_root_dir = '/content/drive/My Drive/BUDT758B_FinalProject/pos/'
neg_root_dir = '/content/drive/My Drive/BUDT758B_FinalProject/neg/'
pos_files = os.listdir(pos_root_dir)
neg_files = os.listdir(neg_root_dir)

corpus = []
labels = []
for i in range(2500):
  f = pos_files[i]
  with open(pos_root_dir+f) as fh:
    corpus.append(fh.read().replace('\n',' '))
    labels.append([1,0])
for i in range(288):
  f = neg_files[i]
  with open(neg_root_dir+f) as fh:
    corpus.append(fh.read().replace('\n',' '))
    labels.append([0,1])


# Remove the punctuation
corpus = [''.join(c for c in s if c not in punctuation) for s in corpus]

# Convert all the letters to lower case.
for i in range(len(corpus)):
  corpus[i] = corpus[i].lower()

# outlier review stats & Determine the max features
review_lens = collections.Counter([len(x) for x in corpus])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))
print("Minimum review length: {}".format(min(review_lens)))
print("Median review length: {}".format(statistics.median(review_lens)))

max_features = 200

# Remove the stop words in English
vectorizer = TfidfVectorizer(max_features = max_features, stop_words = 'english')

X = vectorizer.fit_transform(corpus)
y = np.array(labels)
print(X.shape,y.shape)


Zero-length reviews: 4
Maximum review length: 3520
Minimum review length: 0
Median review length: 317.5
(2788, 200) (2788, 2)


In [ ]:
import torch
from sklearn.model_selection import train_test_split

seq_length = -1

word_tokenizer = vectorizer.build_tokenizer()
vocab = vectorizer.vocabulary_

doc_terms_list_train = [word_tokenizer(doc_str) for doc_str in corpus]
docs = []
for i in range(len(doc_terms_list_train)):
  terms = []
  for j in range(len(doc_terms_list_train[i])):
    w = doc_terms_list_train[i][j]
    if w in vocab:
      terms.append(w)
  if len(terms) > seq_length:
    seq_length = len(terms)
  docs.append(terms)

datasets = np.zeros((X.shape[0],seq_length,max_features))

for i in range(len(docs)):
  # Padding
  n_padding = seq_length - len(docs[i])

  for j in range(len(docs[i])):
    w = docs[i][j]
    idx = vocab[w]
    tfidf_val = X[i,idx]
    datasets[i,j+n_padding,idx] = tfidf_val

datasets = datasets.astype(np.float32)
y = y.astype(np.float32)

X_train,X_val,y_train,y_val = train_test_split(datasets, y, test_size=0.2, random_state=2020)
print(X_train.shape,y_train.shape,X_val.shape,y_val.shape)

(2230, 120, 200) (2230, 2) (558, 120, 200) (558, 2)



### DataLoaders and Batching

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 20

train_data = TensorDataset(torch.from_numpy(X_train),torch.from_numpy(y_train))
val_data = TensorDataset(torch.from_numpy(X_val),torch.from_numpy(y_val))

train_loader = DataLoader(train_data, shuffle=True, batch_size = batch_size)
val_loader = DataLoader(val_data,shuffle=True,batch_size=batch_size)

In [ ]:
from __future__ import unicode_literals, print_function, division

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Model(nn.Module):
  
  def __init__(self, input_size, output_size, hidden_size, n_layers):
    super().__init__()

    self.hidden_size = hidden_size
    self.n_layers = n_layers

# Define three types of layer.
    self.rnn = nn.RNN(input_size,hidden_size,n_layers,batch_first=True)
    self.fc1 = nn.Linear(hidden_size,output_size)
    self.fc2 = nn.Linear(output_size,2)
  
  def forward(self,x,hidden):
    batch_size = x.size()[0]

    hidden = self.init_hidden(batch_size)

    rnn_out,hidden = self.rnn(x,hidden)
    rnn_out = self.fc1(rnn_out)
    last_out = rnn_out[:,-1,:].view(batch_size,-1)
    out = F.softmax(self.fc2(last_out))

    return out,hidden
  
  def init_hidden(self,batch_size):
    hidden = torch.zeros(self.n_layers, batch_size, self.hidden_size).cuda()
    return hidden

model = Model(200,32,256,2)   
print(model) 

Model(
  (rnn): RNN(200, 256, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=256, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=2, bias=True)
)


### Trainning

In [ ]:
# train on Gpu

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
  model.to(device)

# Define hyperparameters
n_epochs = 10
lr = 1e-4
counter = 0
clip = 5

#Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

model.train()
for epoch in range(n_epochs):
  # Initialize hidden state
  h = model.init_hidden(batch_size)

  # Batch loop
  for inputs,labels in train_loader:
    counter +=1
    inputs,labels = inputs.to(device),labels.to(device)

    model.zero_grad()

    outputs,h = model(inputs,h)

    loss = criterion(outputs,torch.max(labels,1)[1])
    loss.backward()

    nn.utils.clip_grad_norm(model.parameters(),clip)
    optimizer.step()

    #Validation loss
    if counter% 10 ==0:
      val_h = model.init_hidden(batch_size).cuda()
      val_losses = []

      model.eval()

      for inputs,labels in val_loader:
        
        inputs, labels = inputs.to(device),labels.to(device)
        val_outputs, val_h = model(inputs,val_h)
        val_loss = criterion(val_outputs,torch.max(labels,1)[1])
        val_losses.append(val_loss.item())

        #pred = torch.round(outputs.squeeze())  # rounds to the nearest integer
    
        # compare predictions to true label
        
        pred = torch.round(val_outputs.squeeze())
        correct_tensor = pred.eq(labels.float().view_as(pred))
        correct = np.squeeze(correct_tensor.cpu().numpy())
        accuracy=np.sum(correct)/2/batch_size
      model.train()

      print('Epoch:{}/{}'.format(epoch+1,n_epochs),
            'Batch:{}'.format(counter),
            'Train Loss:{:.5f}'.format(loss.item()),
            'Val Loss:{:.5f}'.format(np.mean(val_losses)),
            'Accuracy:{:.2%}'.format(accuracy))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch:1/10 Batch:10 Train Loss:0.63751 Val Loss:0.63726 Accuracy:65.00%
Epoch:1/10 Batch:20 Train Loss:0.60843 Val Loss:0.52987 Accuracy:90.00%
Epoch:1/10 Batch:30 Train Loss:0.34245 Val Loss:0.42995 Accuracy:85.00%
Epoch:1/10 Batch:40 Train Loss:0.46558 Val Loss:0.41818 Accuracy:85.00%
Epoch:1/10 Batch:50 Train Loss:0.51393 Val Loss:0.41718 Accuracy:75.00%
Epoch:1/10 Batch:60 Train Loss:0.36476 Val Loss:0.41636 Accuracy:85.00%
Epoch:1/10 Batch:70 Train Loss:0.41419 Val Loss:0.41655 Accuracy:75.00%
Epoch:1/10 Batch:80 Train Loss:0.56335 Val Loss:0.41622 Accuracy:80.00%
Epoch:1/10 Batch:90 Train Loss:0.36417 Val Loss:0.41613 Accuracy:80.00%
Epoch:1/10 Batch:100 Train Loss:0.41388 Val Loss:0.41605 Accuracy:80.00%
Epoch:1/10 Batch:110 Train Loss:0.31416 Val Loss:0.41580 Accuracy:85.00%
Epoch:2/10 Batch:120 Train Loss:0.51347 Val Loss:0.41634 Accuracy:70.00%
Epoch:2/10 Batch:130 Train Loss:0.41373 Val Loss:0.41591 Accuracy:80.00%
Epoch:2/10 Batch:140 Train Loss:0.46357 Val Loss:0.41587 Acc